In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Path to dataset folder in Google Drive
dataset_path = "/content/drive/MyDrive/bone_fracture_dataset/BoneFractureDataset"

# Check if the dataset folder exists
if os.path.exists(dataset_path):
    print("Dataset folder found!")
    print(os.listdir(dataset_path))  # List all files and subfolders
else:
    print("Dataset folder NOT found. Check the path.")

Dataset folder found!
['training', 'testing']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure grayscale
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # Single channel normalization (grayscale)
])


# Load dataset
train_dataset = datasets.ImageFolder(root=dataset_path + "/training", transform=transform)
test_dataset = datasets.ImageFolder(root=dataset_path + "/testing", transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Training samples: 724
Testing samples: 600


In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True

True


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


# Load the pre-trained ResNet model
model = models.resnet34(pretrained=True)

# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Modify the first convolution layer to accept grayscale (1-channel) input
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the fully connected layer to match the number of output classes (2 classes)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Move the model to the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print the modified model to confirm changes
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [ ]:
from sklearn.metrics import accuracy_score

num_epochs = 10
best_accuracy = 0

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    correct = 0
    total = 0
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        _, predicted = torch.max(outputs, 1)  # Get predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()

    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%")

    # Validation
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        print("Saving the best model...")
        torch.save(model.state_dict(), "best_model.pth")  # Save best model


Epoch [1/10], Loss: 0.6634, Train Accuracy: 64.78%
Epoch [1/10], Validation Accuracy: 61.33%
Saving the best model...
Epoch [2/10], Loss: 0.6061, Train Accuracy: 65.47%
Epoch [2/10], Validation Accuracy: 69.33%
Saving the best model...
Epoch [3/10], Loss: 0.5799, Train Accuracy: 68.65%
Epoch [3/10], Validation Accuracy: 66.00%
Epoch [4/10], Loss: 0.5432, Train Accuracy: 71.96%
Epoch [4/10], Validation Accuracy: 62.33%
Epoch [5/10], Loss: 0.5396, Train Accuracy: 73.62%
Epoch [5/10], Validation Accuracy: 59.33%
Epoch [6/10], Loss: 0.5192, Train Accuracy: 75.41%
Epoch [6/10], Validation Accuracy: 63.17%
Epoch [7/10], Loss: 0.5112, Train Accuracy: 76.66%
Epoch [7/10], Validation Accuracy: 56.00%
Epoch [8/10], Loss: 0.5088, Train Accuracy: 76.66%
Epoch [8/10], Validation Accuracy: 62.00%
Epoch [9/10], Loss: 0.4994, Train Accuracy: 75.14%
Epoch [9/10], Validation Accuracy: 60.17%
Epoch [10/10], Loss: 0.5218, Train Accuracy: 77.49%
Epoch [10/10], Validation Accuracy: 60.17%


In [ ]:
import time

start_time = time.time()

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        start_batch_time = time.time()
        outputs = model(images)
        batch_time = time.time() - start_batch_time
        print(f"Batch processing time: {batch_time:.4f} seconds")

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Total Evaluation Time: {time.time() - start_time:.2f} seconds")

Batch processing time: 0.0058 seconds
Batch processing time: 0.0060 seconds
Batch processing time: 0.0058 seconds
Batch processing time: 0.0059 seconds
Batch processing time: 0.0056 seconds
Batch processing time: 0.0057 seconds
Batch processing time: 0.0058 seconds
Batch processing time: 0.0056 seconds
Batch processing time: 0.0059 seconds
Batch processing time: 0.0056 seconds
Batch processing time: 0.0054 seconds
Batch processing time: 0.0056 seconds
Batch processing time: 0.0066 seconds
Batch processing time: 0.0053 seconds
Batch processing time: 0.0061 seconds
Batch processing time: 0.0058 seconds
Batch processing time: 0.0054 seconds
Batch processing time: 0.0068 seconds
Batch processing time: 0.0055 seconds
Test Accuracy: 60.33%
Total Evaluation Time: 5.10 seconds
